<div style="background-color:blue; border-radius:2px; border:#000000 solid; padding: 15px; font-size:100%; text-align:center">
    <h1 align="center" style="color:#ffffff;"><b>Traffic Sign Recognition 🚦🔍</b></h1>
</div>

<div style="background-color:blue; padding:20px; font-size:32px; font-family: consolas; text-align:center; display:fill; border-radius:15px;  color:black;"> <b> 1. Import dependencies</b></div>

In [ ]:
!pip install -q numpy pandas opencv-python matplotlib seaborn tensorflow keras pillow plotly && apt-get update && apt-get install -y libgl1-mesa-glx;

In [ ]:
import numpy as np                 # Importing the NumPy library for numerical computations
import pandas as pd                # Importing the Pandas library for data manipulation and analysis
import os                          # Importing the os module for interacting with the operating system
import cv2                         # Importing the OpenCV library for image processing
import matplotlib.pyplot as plt    # Importing the Matplotlib library for data visualization
import seaborn as sns
import tensorflow as tf            # Importing the TensorFlow library for deep learning
from tensorflow import keras       # Importing the Keras module from TensorFlow for building neural networks
from PIL import Image              # Importing the PIL library for image processing
from tensorflow.keras.optimizers import Adam
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split  # Importing train_test_split from scikit-learn for splitting data
from tensorflow.keras.preprocessing.image import ImageDataGenerator  # Importing ImageDataGenerator for data augmentation
from tensorflow.keras.optimizers import Adam   # Importing the Adam optimizer from TensorFlow for model optimization
from sklearn.metrics import accuracy_score    # Importing accuracy_score from scikit-learn for calculating accuracy
from matplotlib import style       # Importing style from Matplotlib for customizing plot styles
import warnings
warnings.filterwarnings("ignore")  # Ignore warning messages during code execution
np.random.seed(42)                # Setting the random seed for reproducibility

<div style="background-color:blue; padding:20px; font-size:32px; font-family: consolas; text-align:center; display:fill; border-radius:15px;  color:black;"> <b> 2. Data Preparetion</b></div>

In [ ]:
def read_images(directory_path):
    images = []
    labels = []

    valid_extensions = ('.jpg', '.jpeg', '.png')  # Valid image file extensions

    # Get subdirectories within the directory
    subdirectories = [subdir for subdir in os.listdir(directory_path) if os.path.isdir(os.path.join(directory_path, subdir))]

    for subdir in subdirectories:
        subdirectory_path = os.path.join(directory_path, subdir)
        if not os.listdir(subdirectory_path):
            continue  # Skip empty subdirectories

        # Iterate over images in the subdirectory
        for filename in os.listdir(subdirectory_path):
            if filename.lower().endswith(valid_extensions):  # Check if the file has a valid image extension
                image_path = os.path.join(subdirectory_path, filename)
                image = cv2.imread(image_path)
                image = cv2.resize(image, (30, 30))  # Resize the image to 30x30 pixels
                images.append(image)

                # Assign the label based on the subdirectory name
                label = int(subdir)
                labels.append(label)

    data = np.array(list(zip(images, labels)))
    
    return data

In [ ]:
train_data = read_images('/kaggle/input/gtsrb-german-traffic-sign/Train')

In [ ]:
print(f"Train Data Shape : {train_data.shape}")

In [ ]:
# Label Overview
classes = { 
    0:'Speed limit (20km/h)',
    1:'Speed limit (30km/h)', 
    2:'Speed limit (50km/h)', 
    3:'Speed limit (60km/h)', 
    4:'Speed limit (70km/h)', 
    5:'Speed limit (80km/h)', 
    6:'End of speed limit (80km/h)', 
    7:'Speed limit (100km/h)', 
    8:'Speed limit (120km/h)', 
    9:'No passing', 
    10:'No passing veh over 3.5 tons', 
    11:'Right-of-way at intersection', 
    12:'Priority road', 
    13:'Yield', 
    14:'Stop', 
    15:'No vehicles', 
    16:'Veh > 3.5 tons prohibited', 
    17:'No entry', 
    18:'General caution', 
    19:'Dangerous curve left', 
    20:'Dangerous curve right', 
    21:'Double curve', 
    22:'Bumpy road', 
    23:'Slippery road', 
    24:'Road narrows on the right', 
    25:'Road work', 
    26:'Traffic signals', 
    27:'Pedestrians', 
    28:'Children crossing', 
    29:'Bicycles crossing', 
    30:'Beware of ice/snow',
    31:'Wild animals crossing', 
    32:'End speed + passing limits', 
    33:'Turn right ahead', 
    34:'Turn left ahead', 
    35:'Ahead only', 
    36:'Go straight or right', 
    37:'Go straight or left', 
    38:'Keep right', 
    39:'Keep left', 
    40:'Roundabout mandatory', 
    41:'End of no passing', 
    42:'End no passing veh > 3.5 tons'
}

In [ ]:
labels = train_data[:, 1].astype(int)  # Extract the labels from the train_data array
unique_labels, label_counts = np.unique(labels, return_counts=True)
count_dict= {label: count for label, count in zip(unique_labels, label_counts)}
print(count_dict)

In [ ]:
# Define the base color
base_color = '#8A2BE2'  # Blue-violet color

# Create a color scale using the base color
color_scale = [[0, base_color], [1, 'rgb(30,30,30)']]  # Start with the base color and end with dark background

# Create a horizontal bar plot
fig = go.Figure(data=[go.Bar(
    x=label_counts,
    y=[classes[label] for label in unique_labels],
    orientation='h',
    marker=dict(
        color=label_counts,
        coloraxis='coloraxis'
    )
)])

# Customize the layout
fig.update_layout(
    title='Label Counts in Train Data',
    xaxis_title='Count',
    yaxis_title='Label',
    plot_bgcolor='rgb(30,30,30)',  # Dark background color
    paper_bgcolor='rgb(30,30,30)',  # Dark background color
    font=dict(color='white'),  # White font color
    coloraxis=dict(colorscale=color_scale)  # Color scale for bars
)

# Show the plot
fig.show()


In [ ]:
# Define the base color and generate a color palette
base_color = '#8A2BE2'
color_palette = sns.color_palette("husl", len(count_dict)).as_hex()

# Create the pie plot
fig = go.Figure(data=[go.Pie(
    labels=list(count_dict.keys()),
    values=list(count_dict.values()),
    marker=dict(colors=color_palette),
)])

# Update the layout
fig.update_layout(
    title='Label Distribution',
    font=dict(color='white'),
    paper_bgcolor='rgb(30,30,30)',  # Dark background color
    plot_bgcolor='rgb(30,30,30)'  # Dark background color
)

# Show the plot
fig.show()

In [ ]:
# Select 10 random samples from train_data
indices = np.random.choice(train_data.shape[0], size=10, replace=False)
samples = train_data[indices]

# Create a grid of subplots for the images
fig = make_subplots(rows=2, cols=5, subplot_titles=[classes[label] for _, label in samples])

for i, (image, label) in enumerate(samples):
    row = i // 5 + 1
    col = i % 5 + 1

    # Add the image to the subplot
    fig.add_trace(
        go.Image(z=image),
        row=row,
        col=col
    )

# Update the layout
fig.update_layout(
    title='Random Samples',
    font=dict(color='white'),
    paper_bgcolor='rgb(30,30,30)',  # Dark background color
    plot_bgcolor='rgb(30,30,30)'  # Dark background color
)

# Show the plot
fig.show()


In [ ]:
train_ = pd.DataFrame(train_data, columns=['Image', 'Label'])
train_.head()

In [ ]:
class_count = 200  # Number of samples to select from each class
grouped = train_.groupby('Label')# Group the DataFrame by the 'y' column

# Use the apply function to select the desired number of samples from each class
balanced_df = grouped.apply(lambda x: x.sample(class_count))

# Reset the index of the balanced DataFrame
balanced_df.reset_index(drop=True, inplace=True)

In [ ]:
balanced_df.shape

In [ ]:
train_balanced = balanced_df.values

In [ ]:
new_labels = train_balanced[:, 1].astype(int)  # Extract the labels from the train_data array
new_unique_labels, new_label_counts = np.unique(new_labels, return_counts=True)
new_count_dict= {label: count for label, count in zip(new_unique_labels, new_label_counts)}
print(new_count_dict)

In [ ]:
# Define the base color and generate a color palette
base_color = '#8A2BE2'
color_palette = sns.color_palette("husl", len(new_count_dict)).as_hex()

# Create the pie plot
fig = go.Figure(data=[go.Pie(
    labels=list(new_count_dict.keys()),
    values=list(new_count_dict.values()),
    marker=dict(colors=color_palette),
)])

# Update the layout
fig.update_layout(
    title='Label Distribution',
    font=dict(color='white'),
    paper_bgcolor='rgb(30,30,30)',  # Dark background color
    plot_bgcolor='rgb(30,30,30)'  # Dark background color
)

# Show the plot
fig.show()

In [ ]:
shuffled_train = train_balanced.copy()     # Create a copy of train_data to avoid modifying the original array
shuffled_train = np.array(shuffled_train)  # Shuffle the array randomly

In [ ]:
# Split the shuffled_train array into training and testing sets
train_set, test_set = train_test_split(shuffled_train, test_size=0.2, random_state=42)

# Separate the input (X) and output/label (y) arrays from the training set
x_train = train_set[:, 0]
y_train = train_set[:, 1]

# Separate the input (X) and output/label (y) arrays from the testing set
x_test = test_set[:, 0]
y_test = test_set[:, 1]

# Convert the data type of the arrays to int
x_train = np.array([np.array(x) for x in x_train])
y_train = np.array([np.array(y) for y in y_train])
x_test = np.array([np.array(x) for x in x_test])
y_test = np.array([np.array(y) for y in y_test])

# Normalize the input data
x_train = x_train / 255.0
x_test  = x_test / 255.0

print("x_train shape:", x_train.shape)
print("x_test  shape:", x_test.shape)
print("y_train shape:", y_train.shape)
print("y_test  shape:", y_test.shape)

In [ ]:
# Convert target labels to one-hot encoding
y_train = to_categorical(y_train, num_classes=43)
y_test  = to_categorical(y_test, num_classes=43)

<div style="background-color:blue; padding:20px; font-size:32px; font-family: consolas; text-align:center; display:fill; border-radius:15px;  color:black;"> <b> 3. Model Creation and Evaluation</b></div>

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(filters=16, kernel_size=(3, 3), activation='relu', input_shape=(30,30,3)),
    tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2)),
    tf.keras.layers.BatchNormalization(axis=-1),
    
    tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2)),
    tf.keras.layers.BatchNormalization(axis=-1),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(rate=0.5),
    
    tf.keras.layers.Dense(43, activation='softmax')
])

In [ ]:
lr = 0.001
epochs = 30
opt = Adam(learning_rate=lr)  # Set the learning_rate instead of decay
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

aug = ImageDataGenerator(
    rotation_range=15,            # Increase the rotation range for more varied rotations
    zoom_range=0.2,               # Increase the zoom range for more varied zoom levels
    width_shift_range=0.15,       # Increase the range of horizontal shift
    height_shift_range=0.15,      # Increase the range of vertical shift
    shear_range=0.2,              # Increase the shear range for more shearing effects
    horizontal_flip=True,         # Enable horizontal flipping
    vertical_flip=True,           # Enable vertical flipping
    fill_mode="reflect"           # Use reflect mode for filling empty pixels
)


history = model.fit(aug.flow(x_train,y_train, batch_size=32), epochs=epochs, validation_data=(x_test,y_test))

In [ ]:
# Create the figure
fig = go.Figure()

# Add the training loss trace
fig.add_trace(go.Scatter(
    x=list(range(1, epochs+1)),
    y=history.history['loss'],
    mode='lines',
    name='Training Loss'
))

# Add the validation loss trace
fig.add_trace(go.Scatter(
    x=list(range(1, epochs+1)),
    y=history.history['val_loss'],
    mode='lines',
    name='Validation Loss'
))

# Update the layout
fig.update_layout(
    title='Model Loss',
    xaxis_title='Epochs',
    yaxis_title='Loss',
    font=dict(color='white'),
    paper_bgcolor='rgb(30,30,30)',  # Dark background color
    plot_bgcolor='rgb(30,30,30)'  # Dark background color
)

# Show the plot
fig.show()


In [ ]:
# Create the figure
fig = go.Figure()

# Add the training accuracy trace
fig.add_trace(go.Scatter(
    x=list(range(1, epochs+1)),
    y=history.history['accuracy'],
    mode='lines',
    name='Training Accuracy'
))

# Add the validation accuracy trace
fig.add_trace(go.Scatter(
    x=list(range(1, epochs+1)),
    y=history.history['val_accuracy'],
    mode='lines',
    name='Validation Accuracy'
))

# Update the layout
fig.update_layout(
    title='Model Accuracy',
    xaxis_title='Epochs',
    yaxis_title='Accuracy',
    font=dict(color='white'),
    paper_bgcolor='rgb(30,30,30)',  # Dark background color
    plot_bgcolor='rgb(30,30,30)'  # Dark background color
)

# Show the plot
fig.show()


In [ ]:
# Convert NumPy array to DataFrame
shuffled_train_dataframe = pd.DataFrame(shuffled_train)
# Save DataFrame as CSV file
shuffled_train_dataframe.to_csv('data.csv', index=False)

In [ ]:
model.save("saved_model.h5")

In [ ]:
def predict_image(model_path, image_path):
    # Load the model
    model = tf.keras.models.load_model(model_path)
    
    # Load and preprocess the image
    image = cv2.imread(image_path)
    image = cv2.resize(image, (30, 30))
    image = np.expand_dims(image, axis=0)
    image = image / 255.0
    
    # Make predictions
    predictions = model.predict(image)
    predicted_label = np.argmax(predictions)
    
    return predicted_label

model_path = "/kaggle/working/saved_model.h5"
image_path = "/kaggle/input/gtsrb-german-traffic-sign/Test/00080.png"

predicted_label = predict_image(model_path, image_path)
print("Predicted Label:", classes[predicted_label])


<div style="background-color:blue; padding:20px; font-size:32px; font-family: consolas; text-align:center; display:fill; border-radius:15px;  color:black;"> <b> 4. Conclusion </b></div>

## **Conclusion**
The traffic sign recognition model notebook provides a comprehensive analysis and prediction model for recognizing traffic signs. By using deep learning techniques and the provided dataset, the notebook demonstrates how to preprocess the images, build a convolutional neural network (CNN) model, and evaluate its performance using accuracy as the metric. The notebook also showcases the effectiveness of data augmentation techniques in improving the model's performance.

The CNN model architecture consists of several convolutional and pooling layers, followed by fully connected layers and a softmax activation function for multi-class classification. The model is trained using the Adam optimizer and categorical cross-entropy loss function.

By training the model on the traffic sign dataset, the notebook demonstrates how to achieve high accuracy in recognizing and classifying different traffic signs. The model can be used in real-world applications to aid in automated traffic sign detection and recognition systems.

Overall, this notebook serves as a valuable resource for understanding and implementing traffic sign recognition using deep learning techniques.

## **Licenses and Social Media Accounts**

### **Licenses:**

The code and content in this notebook are released under the following licenses:

- The code is licensed under the [MIT License](https://opensource.org/licenses/MIT).
- The content, including text and images, is licensed under the Creative Commons Attribution 4.0 International License ([CC BY 4.0](https://creativecommons.org/licenses/by/4.0/)).

### **Social Media Accounts:**

**Connect with me on social media to stay updated with the latest news, articles, and projects:**

- LinkedIn  : [@thisishusseinali](https://www.linkedin.com/in/thisishusseinali)
- GitHub   : [@thisishusseinali](https://github.com/thisishusseinali)
- Instagram : [@thisishusseinali](https://www.instagram.com/thisishusseinali)
- Medium    : [@thisishusseinali](https://medium.com/@thisishusseinali)

**Feel free to reach out to me on any of these platforms. I would love to hear your feedback and engage in discussions related to data science, machine learning, and more!**
